In [ ]:
!pip install pytorch-forecasting==1.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.9/827.9 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 51.7 MB/s eta 0:00:00


In [ ]:
!pip install torch

In [ ]:
!pip install torchrl pandas numpy matplotlib seaborn tqdm yfinance pytorch-forecasting==1.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.0/450.0 kB 27.6 MB/s eta 0:00:00


In [ ]:
!pip install pytorch_forecasting

In [ ]:
#pip install --upgrade torchrl tensordict

In [ ]:
import torchrl
print(f"目前使用的 torchrl 版本是: {torchrl.__version__}")

目前使用的 torchrl 版本是: 0.0.0+unknown


In [ ]:
import os, warnings, random, numpy as np, pandas as pd, yfinance as yf
import torch, torch.nn as nn
from tqdm import tqdm

warnings.filterwarnings("ignore"); torch.set_float32_matmul_precision("medium")

# ------------------------------------------------------------------ 0.  deps
# pip install torch torchrl pandas numpy matplotlib seaborn tqdm yfinance pytorch-forecasting==1.5.0
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from torchrl.envs import EnvBase, TransformedEnv, Compose, StepCounter, RewardScaling
from torchrl.data import BoundedTensorSpec, CompositeSpec # Import CompositeSpec
from torchrl.modules import ProbabilisticActor, TanhNormal
from tensordict.nn import TensorDictModule # Corrected import for TensorDictModule
from torchrl.objectives import SACLoss
from torchrl.collectors import SyncDataCollector
from tensordict import TensorDict
#from pytorch_forecasting.models.temporal_fusion_transformer import TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer # Corrected typo here

# ---- debug probe ----
import lightning.pytorch as pl, pytorch_forecasting as pf # Changed import to lightning.pytorch
from pytorch_forecasting.models.temporal_fusion_transformer import TemporalFusionTransformer
print("PF ver :", pf.__version__)
print("PL ver :", pl.__version__)
print("TFT MRO:", TemporalFusionTransformer.__mro__)
# ---------------------


SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ------------------------------------------------------------------ 1.  download daily data
TICKER = "AAPL"
df = yf.download(TICKER, start="2016-01-01", end="2024-01-01")["Close"]
df.columns = ["price"]
df["ret"] = df.price.pct_change()
df["log_ret"] = np.log1p(df.ret)

# -------------------- 2.  fake quantamental & macro -----------------
np.random.seed(SEED)
df["pe"]   = np.random.lognormal(2.2, 0.25, len(df))
df["roe"]  = np.random.normal(0.15, 0.04, len(df))
df["rsi"]  = df.ret.rolling(14).mean() / df.ret.rolling(14).std()
df["usd"]  = np.sin(np.arange(len(df)) * 2 * np.pi / 252)
df["vol"]  = df.ret.rolling(20).std()
df["spread"] = df.price * 0.0005   # 5 bp proxy
df = df.dropna().reset_index()
df["ticker"] = TICKER
df["time_idx"] = (df["Date"] - df["Date"].min()).dt.days
TARGET = "log_ret"
print("rows:", len(df))

# -------------------- 3.  build TimeSeriesDataSet -------------------
max_enc, max_pred = 60, 1
training_cutoff = df.time_idx.max() - 252   # last year for validation

training = TimeSeriesDataSet(
    df[df.time_idx <= training_cutoff],
    time_idx="time_idx",
    target=TARGET,
    group_ids=["ticker"],
    min_encoder_length=max_enc//2,
    max_encoder_length=max_enc,
    min_prediction_length=max_pred,
    max_prediction_length=max_pred,
    static_categoricals=["ticker"],
    time_varying_known_reals=["time_idx", "usd"],
    time_varying_unknown_reals=["price", "rsi", "pe", "roe", "ret", "vol"],
    target_normalizer=GroupNormalizer(groups=["ticker"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    allow_missing_timesteps=True,
)
validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)
train_loader = training.to_dataloader(batch_size=128, shuffle=True)
val_loader   = validation.to_dataloader(batch_size=256, shuffle=False)

from torchmetrics import MeanAbsoluteError
import lightning.pytorch as pl # Changed import to lightning.pytorch

# -------------------- 4.  train TFT ---------------------------------

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=1e-3,
    hidden_size=64,
    attention_head_size=2,
    dropout=0.1,
    #loss=MeanAbsoluteError(),
    #logging_metrics=["MAE"],
    reduce_on_plateau_patience=3,
)
print("is LM ?", isinstance(tft, pl.LightningModule))
#print(tft)

trainer = pl.Trainer(
    max_epochs=1,
    accelerator=DEVICE,
    gradient_clip_val=0.1,
    enable_progress_bar=True,
    enable_model_summary=False,
    logger=False # Added to suppress INFO messages
)

trainer.fit(tft, train_loader, val_loader)

# -------------------- 5.  add TFT forecast & vol to df --------------
def add_tft_mu_sigma(df, tft, dataset):
    enc_len = dataset.max_encoder_length
    max_pred = dataset.max_prediction_length # This should be 1

    # Create a prediction dataset for the entire dataframe,
    # ensuring it generates samples for all possible prediction points.
    # We use the same parameters as the original `training` dataset.
    full_prediction_dataset = TimeSeriesDataSet(
        df, # Use the full dataframe
        time_idx="time_idx",
        target=TARGET,
        group_ids=["ticker"],
        min_encoder_length=enc_len // 2, # Same as training
        max_encoder_length=enc_len,     # Same as training
        min_prediction_length=max_pred, # Always predict one step ahead
        max_prediction_length=max_pred, # Always predict one step ahead
        static_categoricals=["ticker"],
        time_varying_known_reals=["time_idx", "usd"],
        time_varying_unknown_reals=["price", "rsi", "pe", "roe", "ret", "vol"],
        target_normalizer=dataset.target_normalizer, # Use the fitted normalizer from the training dataset
        add_relative_time_idx=True,
        add_target_scales=True,
        allow_missing_timesteps=True,
    )

    # Create a DataLoader for this prediction dataset
    pred_loader = full_prediction_dataset.to_dataloader(batch_size=256, shuffle=False)

    # Predict
    with torch.no_grad():
        predictions = tft.predict(pred_loader, return_x=True, return_y=False)

    # predictions will be a tuple: (predicted_mean, x_input)
    predicted_mean = predictions[0].flatten().cpu().numpy()

    # The `x_input` (predictions[1]) contains the input TensorDict.
    # We need to map the predicted means back to the correct `time_idx` in `df`.
    # `decoder_time_idx` in `x_input` gives the time indices corresponding to the predictions.
    predicted_time_indices = predictions[1]["decoder_time_idx"].flatten().cpu().numpy()

    # Create a temporary DataFrame from predictions
    pred_df_raw = pd.DataFrame({
        'time_idx': predicted_time_indices,
        'mu_pred': predicted_mean
    })

    # Aggregate by time_idx to ensure uniqueness, taking the mean of 'mu_pred' if duplicates exist
    # This step is crucial to handle the InvalidIndexError
    pred_df_unique = pred_df_raw.groupby('time_idx')['mu_pred'].mean().reset_index()

    # Create a Series for mapping with a unique index
    pred_series = pred_df_unique.set_index('time_idx')['mu_pred']

    # Initialize 'mu' column with NaNs, then fill in predictions
    df["mu"] = np.nan
    df["mu"] = df["time_idx"].map(pred_series)
    df["sig"] = df["vol"]
    return df.dropna(subset=["mu"]) # Drop rows where mu could not be predicted (initial max_encoder_length)

rl_df = add_tft_mu_sigma(df, tft, training)

# -------------------- 6.  TorchRL environment -----------------------
class TFTTradingEnv(EnvBase):
    def __init__(self, df, cost_bp=8, risk_pen=1e-4):
        super().__init__()
        self.df  = df.reset_index(drop=True)
        self.k   = cost_bp/1e4
        self.lam = risk_pen
        self._pos, self._t = 0.0, 0
        # Wrap BoundedTensorSpec within a CompositeSpec for state_spec
        self.state_spec  = CompositeSpec(observation=BoundedTensorSpec(low=-torch.inf, high=torch.inf, shape=(6,)))
        self.action_spec = BoundedTensorSpec(low=-1.0, high=1.0, shape=(1,))

    def _reset(self, tensordict=None):
        self._t, self._pos = 0, 0.0
        return TensorDict({"observation": self._make_obs()}, batch_size=[])

    def _make_obs(self):
        row = self.df.iloc[self._t]
        obs = torch.tensor([row.mu, row.sig, self._pos,
                            row.ret, row.vol, row.spread], dtype=torch.float32)
        return obs

    def _step(self, tensordict):
        action = tensordict["action"].clip(-1, 1).item()
        row    = self.df.iloc[self._t]
        ret    = row.ret
        reward = self._pos * ret - self.k * abs(action - self._pos) \
                 - self.lam * (self._pos * row.vol)**2
        self._pos = action
        self._t  += 1
        done = self._t >= len(self.df)-1
        obs  = self._make_obs()
        return TensorDict({
            "observation": obs,
            "reward": torch.tensor(reward, dtype=torch.float32).view(1),
            "done": torch.tensor(done).view(1),
        }, batch_size=[])

    def _set_seed(self, seed): torch.manual_seed(seed)

# -------------------- 7.  SAC training ----------------------------
env = TFTTradingEnv(rl_df)
env = TransformedEnv(env, Compose(StepCounter(500), RewardScaling(loc=0.0, scale=10.0)))

actor_net = nn.Sequential(nn.Linear(6,64), nn.Tanh(),
                          nn.Linear(64,64), nn.Tanh(),
                          nn.Linear(64,2) # Output 2 for loc and scale
                         ).to(DEVICE)

# Wrap actor_net in TensorDictModule to specify in_keys and out_keys explicitly
actor_module = TensorDictModule(
    actor_net,
    in_keys=["observation"],
    out_keys=["loc", "scale"] # TanhNormal expects loc and scale
)

actor = ProbabilisticActor(
    module=actor_module,
    in_keys=["observation"], # Inputs to the actor_module
    distribution_class=TanhNormal,
    # Explicitly define which outputs from actor_module are distribution parameters
    dist_keys=["loc", "scale"],
    # Pass distribution_kwargs to TanhNormal for action bounds
    distribution_kwargs={
        "low": env.action_spec.low,
        "high": env.action_spec.high,
    },
    return_log_prob=True
).to(DEVICE)

q1_net = nn.Sequential(nn.Linear(6+1,64), nn.Tanh(),
                   nn.Linear(64,64), nn.Tanh(),
                   nn.Linear(64,1)).to(DEVICE)
q1 = TensorDictModule(q1_net, in_keys=["observation", "action"], out_keys=["q_value"])

q2_net = nn.Sequential(nn.Linear(6+1,64), nn.Tanh(),
                   nn.Linear(64,64), nn.Tanh(),
                   nn.Linear(64,1)).to(DEVICE)
q2 = TensorDictModule(q2_net, in_keys=["observation", "action"], out_keys=["q_value"])

# Pass the action_spec and the list of q_value_nets to SACLoss
# Change to pass q1 and q2 as separate positional arguments
loss_mod = SACLoss(
    actor,
    q1,
    q2,
    action_spec=env.action_spec,)
opt = torch.optim.Adam(loss_mod.parameters(), lr=3e-4)

collector = SyncDataCollector(env, policy=actor, frames_per_batch=2_000,
                              total_frames=100_000, device=DEVICE)

pbar = tqdm(collector)
for batch in pbar:
    loss_vals = loss_mod(batch)
    opt.zero_grad()
    loss_vals["loss"].backward()
    opt.step()
    pbar.set_postfix(loss=loss_vals["loss"].item())

# -------------------- 8.  evaluate -------------------------------
env.eval()
with torch.no_grad():
    roll = env.rollout(max_steps=len(rl_df)-1, policy=actor, auto_reset=True)
net_ret = roll["reward"].sum().item()
print("Net P&L over full history:", net_ret)

PF ver : 1.5.0
PL ver : 2.5.6
TFT MRO: (<class 'pytorch_forecasting.models.temporal_fusion_transformer._tft.TemporalFusionTransformer'>, <class 'pytorch_forecasting.models.base._base_model.BaseModelWithCovariates'>, <class 'pytorch_forecasting.models.base._base_model.BaseModel'>, <class 'pytorch_forecasting.utils._utils.InitialParameterRepresenterMixIn'>, <class 'lightning.pytorch.core.module.LightningModule'>, <class 'lightning.fabric.utilities.device_dtype_mixin._DeviceDtypeModuleMixin'>, <class 'lightning.pytorch.core.mixins.hparams_mixin.HyperparametersMixin'>, <class 'lightning.pytorch.core.hooks.ModelHooks'>, <class 'lightning.pytorch.core.hooks.DataHooks'>, <class 'lightning.pytorch.core.hooks.CheckpointHooks'>, <class 'torch.nn.modules.module.Module'>, <class 'pytorch_forecasting.utils._utils.TupleOutputMixIn'>, <class 'object'>)


[*********************100%***********************]  1 of 1 completed


rows: 1992


INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores


is LM ? True


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.12/dist-packages/torchrl/data/tensor_specs.py:7085: DeprecationWarning: The BoundedTensorSpec has been deprecated and will be r

TypeError: SafeProbabilisticModule.__init__() got an unexpected keyword argument 'dist_keys'